In [3]:
# Please download the dataset and save it in the 'data' folder.
# The folder structure should be as follows:
# ./data/
# ├── test/
# │   ├── को/
# │   │   └── को_5.jpg  # Each image is in a folder with the character label.



# ├── train/
# │   ├── को/
# │   │   └── को_1.jpg  # Each image is in a folder with the character label.

print('TRaiFIC : ANPR')


TRaiFIC : ANPR


In [4]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader


transform = transforms.Compose([
    transforms.Grayscale(),  
    transforms.Resize((32, 32)),  
    transforms.ToTensor(),  
    transforms.Normalize((0.5,), (0.5,))  
])


train_dataset = ImageFolder(root='data/train', transform=transform)
test_dataset = ImageFolder(root='data/test', transform=transform)


batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


print("Class labels:", train_dataset.classes)
print("End  loaded dataset")


Class labels: ['क', 'को', 'ख', 'ग', 'च', 'ज', 'झ', 'ञ', 'डि', 'त', 'ना', 'प', 'प्र', 'ब', 'बा', 'भे', 'म', 'मे', 'य', 'लु', 'सी', 'सु', 'से', 'ह', '०', '१', '२', '३', '४', '५', '६', '७', '८', '९']
End  loaded dataset


In [5]:
import torch.nn as nn
import torch.nn.functional as F

class NepaliPlateCNN(nn.Module):
    def __init__(self, num_classes):
        super(NepaliPlateCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)  # Adjust based on input size
        self.fc2 = nn.Linear(512, num_classes)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Output: 32x16x16
        x = self.pool(F.relu(self.conv2(x)))  # Output: 64x8x8
        x = self.pool(F.relu(self.conv3(x)))  # Output: 128x4x4
        x = x.view(-1, 128 * 4 * 4)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model
num_classes = len(train_dataset.classes)
model = NepaliPlateCNN(num_classes)
print(model)
print("End")


NepaliPlateCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=2048, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=34, bias=True)
)
End


In [6]:
import torch.optim as optim

# Define loss function and optimizer
# Using CrossEntropyLoss for multi-class classification
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Check if GPU is available and move the model to GPU if possible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Training loop
num_epochs = 2
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # Print epoch loss
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

print("Training complete!")
print("End")


Epoch [1/2], Loss: 0.5917
Epoch [2/2], Loss: 0.0892
Training complete!
End


In [7]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 97.41%


In [9]:
# Save the trained model
torch.save(model.state_dict(), "char_traiffic_model_beta_v1.pth")

# Load the model for inference
loaded_model = NepaliPlateCNN(num_classes)
loaded_model.load_state_dict(torch.load("char_traiffic_model_beta_v1.pth"))
loaded_model.eval()

NepaliPlateCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=2048, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=34, bias=True)
)

In [ ]:
import cv2
import numpy as np
import torch
import os

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image not found at {image_path}")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (32, 32))
    image = image / 255.0  # Normalize to [0, 1]
    image = torch.FloatTensor(image).unsqueeze(0).unsqueeze(0)  # Add batch and channel dims
    return image

# Define device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the trained model
num_classes = len(train_dataset.classes)  # Ensure train_dataset is loaded
loaded_model = NepaliPlateCNN(num_classes)
loaded_model.load_state_dict(torch.load("char_traiffic_model_beta_v1.pth"))
loaded_model.eval()
loaded_model.to(device)

# Example usage
# image_path = "./सु_409.jpg"  # Replace with the correct path to your image

image_path = os.path.abspath(r"input_files/na.jpg")
try:
    input_image = preprocess_image(image_path).to(device)
    output = loaded_model(input_image)
    _, predicted = torch.max(output.data, 1)
    print("Predicted class:", train_dataset.classes[predicted.item()])
except FileNotFoundError as e:
    print(e)

Predicted class: ना
